In [1]:
!git clone https://github.com/WongKinYiu/yolov7 
!cd yolov7 && wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
%pip install imagesize
%pip install ptitprince

# %pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import json
import sys
import os
import shutil
from distutils.dir_util import copy_tree
import imagesize
import ptitprince as pt
from shutil import copyfile
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patches as mpatches
import seaborn as sns
import re
import xml.etree.ElementTree as ET 
from sklearn.model_selection import train_test_split



ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [3]:
cd /kaggle/working/

[Errno 2] No such file or directory: '/kaggle/working/'
/home/gzm/projects/AI_Chatbot_PCB_Detection


In [ ]:
try:
    os.listdir('./dest')
except:
    pass

## 데이터 프레임 생성

In [ ]:
!cd /kaggle/working/

path_an = "/kaggle/input/pcb-defects/PCB_DATASET/Annotations"

dataset = {
            "class":[],
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "width":[],
            "height":[],
            "file":[],
            "defect_x_center":[],
            "defect_y_center":[],
            "defect_width":[],
            "defect_height":[],
            "image_width":[],
            "image_height":[],
            "path":[],
            "category_codes":[]
           }
all_files = []
for path, subdirs, files in os.walk(path_an):
#     print([path, subdirs, files])
    for name in files:
        all_files.append(os.path.join(path, name))
        
        

for anno in all_files:
    # print(anno)
    tree = ET.parse(anno)
    
    
    cnt = 0
    for elem in tree.iter():
        if 'size' in elem.tag:
            # print('[size] in elem.tag ==> list(elem)\n'), print(list(elem))
            for attr in list(elem):
                if 'width' in attr.tag: 
                    image_width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    image_height = int(round(float(attr.text)))

        if 'object' in elem.tag:
            cnt += 1
            # print('[object] in elem.tag ==> list(elem)\n'), print(list(elem))
            for attr in list(elem):
                
                # print('attr = %s\n' % attr)
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['class']+=[name]
                    
                    if name == 'missing_hole':
                        dataset['category_codes'] += [1]
                    elif name == 'mouse_bite':
                        dataset['category_codes'] += [2]
                    elif name == 'open_circuit':
                        dataset['category_codes'] += [3]
                    elif name == 'short':
                        dataset['category_codes'] += [4]
                    elif name == 'spur':
                        dataset['category_codes'] += [5]
                    elif name == 'spurious_copper':
                        dataset['category_codes'] += [6]
                    
                    dataset['image_width']+=[image_width]
                    dataset['image_height']+=[image_height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    xmin = -1 
                    ymin = -1
                    xmax = -1
                    ymax = -1
                    for dim in list(attr):
                        #print('-', dim.tag)
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]
                    width = xmax - xmin
                    height = ymax - ymin
                    dataset['width'] += [width]
                    dataset['height'] += [height]
                    dataset['defect_x_center'] += [(((xmin + width) + xmin) / 2) / image_width]
                    if((((xmin + width) + xmin) / 2) / image_width >= 1):
                        print(xmin, ' + ', width, ' + ', xmin, ' / ', 2, ' / ', image_width)
                    dataset['defect_y_center'] += [(((ymin + height) + ymin) / 2) / image_height]
                    dataset['defect_width'] += [width / image_width]
                    dataset['defect_height'] += [height / image_height]
                    
                    
                            
    
    for i in range(cnt):
        tmp_path = tree.find('path').text
        #print(tmp_path)
        dataset['path']+= ['/kaggle/input/pcb-defects/PCB_DATASET/images' + tmp_path[32:]]


In [ ]:
anno = all_files[0]
tree = ET.parse(anno)
tmp_path = tree.find('path').text
print(tmp_path.rfind('/'))
print(tmp_path[44:])

In [ ]:
for i in dataset.keys():
    print(i, len(dataset[i]))

In [ ]:
try:
    os.makedirs('./dest', exist_ok=True)
except:
    print('no new directory was made, probably already existing')
    


In [ ]:
print(os.getcwd())
df = pd.DataFrame(dataset)

defect = df['class'].unique()

df_1 = df[df['class'] == defect[0]]

for i in defect:
    tmp_df = df[df['class'] == i]
    tmp_df.to_csv('./dest/annotation_' + i + '.csv')

In [ ]:
df = pd.read_csv('./dest/annotation_' + defect[0] + '.csv')
idx = df['path'].unique()
type(idx)
df[df['path'].isin(df['path'].unique())]

## 객체 생성

In [ ]:
!cd /kaggle/working/
class DataProcess():
    def __init__(self, data, save_dest, classes, all_path):
        self.data = data # DataFrame
        self.save_dest = save_dest
        self.classes = classes
        self.all_path = all_path
        
    
    def split(self):     # 5
        train_file, test_file = train_test_split(self.data['path'].unique(), test_size = 0.2)
        self.train_data = self.data[self.data['path'].isin(train_file)]
        self.test_data = self.data[self.data['path'].isin(test_file)]
        self.train_data.sort_values('path')
        self.test_data.sort_values('path')
        #print(self.train_data)
    
    def make_image_path_txt(self):    # 1
        with open(self.save_dest + '/train.txt', "w") as f:
            for path in self.train_data['path'].unique():
                print(path, file = f)
        
        with open(self.save_dest + '/test.txt', 'w') as f:
            for path in self.test_data['path'].unique():
                print(path, file = f)
                 
    def make_dir(self):    # 
        try:
            os.makedirs(self.save_dest, exist_ok=True)
        except:
            print('no new directory was made, probably already existing')
        
    
    def reset_all(self):    # 1
        os.removedirs('/kaggle/working/dest')
    
    def image_copy_all(self):    # 2
        self.make_dir()
        try:
            os.makedirs('./dest/images')
        except:
            pass
        

        #for i in self.classes:
            #for k in self.data['path']:
                #idx = k.rfind('/')
                #idx += 1
                #copyfile(k, os.path.join(self.save_dest, 'images', k[idx:]))
                
        for i in range(len(self.data)):
            index = self.data['path'][i].rfind('/')
            index += 1
            file_name = self.data['path'][i]
            copyfile(self.data['path'][i], os.path.join(self.save_dest, 'images', file_name[index:]))
            self.data['path'][i] = os.path.join(self.save_dest, 'images', file_name[index:])
            
    
    def make_label(self):     # 3
        self.make_dir()
        
        try:
            os.removedirs('/kaggle/working/dest/labels/')
            print('erase success')
        except:
            print("can't erase labels")
        
        try:
            os.makedirs('/kaggle/working/dest/labels/')
        except:
            print("creat fail")
        
        
        for i in range(0, len(self.data)):
            #print(self.data['path'][i])
            index = self.data['path'][i].rfind('/')
            
            index += 1
            file_name = self.data['path'][i]
            #print(os.path.join(self.save_dest,'labels', file_name[index:-4] + '.txt'))
            
            try:
                with open(os.path.join(self.save_dest,'labels', file_name[index:-4] + '.txt'), 'a') as f:
                    #print(os.path.join(self.save_dest,'labels', file_name[index:-4] + '.txt'))
                    print(self.data['category_codes'][i], self.data['defect_x_center'][i], self.data['defect_y_center'][i], self.data['defect_width'][i], self.data['defect_height'][i], file = f)
            except:
                print('failed label creating')
        
    def make_yaml(self):     # 4
        with open(self.save_dest + '/train.yaml', 'w') as f:
            print('yaml:', file = f)
            
            classes = "[ 'Nothing'"
            for i in self.classes:
                classes += ', ' + "'" + i + "'"  
            classes += ']'
            
            print('names: ' + classes, file=f)
            print('nc:', str(len(self.classes) + 1) , file = f)
            print('train:  ', self.all_path + '/train.txt', file = f)
            #print(self.all_path + '/train.txt')
            print('val:  ', self.all_path + '/test.txt', file = f)
            #print('val:  ', self.all_path + '/test.txt')
    
    def check_yaml(self):
        with open(self.save_dest + '/train.yaml', 'r') as f:
            print(f.readlines())

## 시각화

In [ ]:
def visualisation_annotations(dat, filedir, fname):
    """
    dat: input pandas DataFrame
    filedir: directory where image files can be found
    fname: filename to be visualized, for ex 'p1.JPG'  
    """
    im = Image.open(os.path.join(filedir, fname))
    fig, ax = plt.subplots(figsize=(14, 20))
    ax.imshow(im)
    ndat = dat[dat['path'] == os.path.join(filedir, fname)].reset_index()
        
    for i in range(len(ndat)):    
        xmin = ndat['xmin'][i]
        ymin = ndat['ymin'][i]
        w = ndat['width'][i] # bb_width
        h = ndat['height'][i] # bb_w
        #color = ndat['color_cat'][i]
        rect = patches.Rectangle((xmin, ymin), w, h, linewidth=2, edgecolor='red', facecolor='none') # takes x, y, width and height.
        ax.add_patch(rect)
        
    #mpatches_dat = pd.DataFrame(df[['classes'].value_counts().reset_index())
     #   patch1 = mpatches.Patch(color=mpatches_dat['color_cat'][0], label=mpatches_dat['category_names'][0])
   # patch2 = mpatches.Patch(color=mpatches_dat['color_cat'][1], label=mpatches_dat['category_names'][1])
   # ax.legend(handles=[patch1])
        
    plt.show()

## 객체 생성 및 사용 테스트

In [ ]:
data_path = []
for i in defect:
    data_path.append('./dest/annotation_' + i + '.csv')

print(data_path)
data = DataProcess(pd.read_csv(data_path[4]), '/kaggle/working/dest', [defect[0]], '/kaggle/working/dest')

In [ ]:
data.image_copy_all()
data.data

In [ ]:
#data.reset_all()
data.make_label()
data.make_yaml()
data.split()
data.make_image_path_txt()

In [ ]:
!cat '/kaggle/working/dest/train.yaml'

In [ ]:
visualisation_annotations(data.data,'/kaggle/working/dest/images', '09_missing_hole_02.jpg')

In [ ]:
lst = os.listdir('/kaggle/working/dest/labels')
lst.sort()
lst

In [ ]:
f = open('/kaggle/working/dest/train.yaml')
a = f.readlines()
a

In [ ]:
!cp './yolov7/cfg/training/yolov7.yaml' '/kaggle/working/yolov7_ad.yaml'
with open("/kaggle/working/yolov7_ad.yaml") as r:
  text = r.read().replace("nc: 80", "nc: 2")
with open("/kaggle/working/yolov7_ad.yaml", "w") as w:
  w.write(text)


In [ ]:
cd /kaggle/working/yolov7/

In [ ]:
!cp 'data/hyp.scratch.custom.yaml' '/kaggle/working/hypownsettings.yaml' 
with open("/kaggle/working/hypownsettings.yaml") as r:
  text = r.read().replace("mosaic: 1.0", "mosaic: 0.0").replace("flipud: 0.0", "flipud: 0.5").replace("scale: 0.5", "scale: 0.0")
with open("/kaggle/working/hypownsettings.yaml", "w") as w:
  w.write(text)

In [ ]:
# !git clone https://github.com/WongKinYiu/yolov7 
# !cd yolov7 && wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!pip install wandb

In [ ]:
!grep -w init_seeds 'yolov7/utils/general.py' -A 4 

In [ ]:
os.environ["WANDB_MODE"]="offline"

In [ ]:
with open('/kaggle/working/dest/train.yaml', 'r') as f:
    print(f.readlines())

In [ ]:
print(os.listdir('/kaggle/working/dest/'))
try:
    os.remove('/kaggle/working/dest/train.cache')
except:
    print('fail')

In [ ]:
data.make_yaml()
!cat /kaggle/working/dest/train.yaml

In [ ]:
!cd /kaggle/working/yolov7/
!python train.py --img 640 --batch 16 --epochs 100 --data '/kaggle/working/dest/train.yaml' --hyp "/kaggle/working/hypownsettings.yaml" --image-weights --cfg "/kaggle/working/yolov7_ad.yaml" --weights '/kaggle/working/yolov7.pt' --cache

In [ ]:
with open('/kaggle/working/dest/train.yaml', 'r') as f:
    print(f.readlines())

In [ ]:
# with open('/kaggle/working/dest/train.txt', 'r') as f:
#     print(f.readlines())

f = pd.read_table('/kaggle/working/dest/train.txt', sep = ',')
f

In [ ]:
# with open('/kaggle/working/dest/test.txt', 'r') as f:
#     print(f.readlines())
f = pd.read_table('/kaggle/working/dest/test.txt', sep = ',')
f

In [ ]:
print(os.listdir('../yolov7/runs/train/exp2'))

im = Image.open('../yolov7/runs/train/exp2/test_batch0_pred.jpg')
im

In [ ]:

f = pd.read_table('/kaggle/working/dest/train.yaml')
f